In [68]:
import pandas
import os
from tqdm import tqdm

In [114]:
def process_scenario(root_folder, year, era5_year, region):
    df_current_scenario = pandas.read_csv(f"{root_folder}/{year}/{era5_year}/{region}", delimiter=";")

    df_current_scenario["era5 source year"] = era5_year.split("_")[-1]
    df_current_scenario["year"] = year

    # Namibia bug
    df_current_scenario.loc[df_current_scenario["region_name"] == "Namibia", "region_code"] = "NA"

    df_current_scenario.sort_values(["region_code", "time"], inplace=True)

    df_current_scenario['datetime'] = pandas.to_datetime(df_current_scenario['time'])
    df_current_scenario['hour_of_year'] = (df_current_scenario['datetime'].dt.dayofyear - 1) * 24 + df_current_scenario['datetime'].dt.hour

    df_current_scenario.drop(columns=["time", "datetime", "region_name"], inplace=True)

    return df_current_scenario

In [116]:
root_folder = "./comparison/GEGIS/ssp2-2.6/"
df_all_scenarios = pandas.DataFrame()

for year in os.listdir(root_folder):
    for era5_year in os.listdir(os.path.join(root_folder, year)):
        for region in os.listdir(os.path.join(root_folder, year, era5_year)):
            if region.endswith(".csv"):
                df_all_scenarios = pandas.concat([df_all_scenarios, process_scenario(root_folder, year, era5_year, region)])

In [119]:
df_all_scenarios.to_parquet("./comparison/GEGIS/ssp2-2.6_processed.parquet")

In [121]:
pandas.read_parquet("./comparison/GEGIS/ssp2-2.6_processed.parquet")

,region_code,Electricity demand,era5 source year,year,hour_of_year
0,AO,1288.203173,2018,2030,0
1,AO,1308.701208,2018,2030,1
2,AO,1315.373287,2018,2030,2
3,AO,1335.551572,2018,2030,3
4,AO,1521.880371,2018,2030,4
...,...,...,...,...,...
122635,VE,28137.441009,2013,2050,8755
122636,VE,28137.441009,2013,2050,8756
122637,VE,28282.094003,2013,2050,8757
122638,VE,28787.690805,2013,2050,8758


In [122]:
df_current_scenario.dtypes

region_code            object
Electricity demand    float64
era5 source year        int64
year                    int64
hour_of_year            int32
dtype: object